In [1]:
import pandas as pd

In [2]:
#Defining the file to load
path="../Resources/purchase_data.csv"

#opening the data and displaying first 5 rows of the table
purchase_data_df=pd.read_csv(path)
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#finding the number of unique SN to get the number of players
players=len(purchase_data_df["SN"].unique())
#displaying total number of players as data frame
total_players_df=pd.DataFrame({"Total Players":[players]})
total_players_df

,Total Players
0,576


In [4]:
#running basic calculations 
items=purchase_data_df["Item Name"].nunique()
average_price=purchase_data_df["Price"].mean()
purchases_number = len(purchase_data_df["Purchase ID"])
revenue=purchase_data_df["Price"].sum()
#creating a summary table to hold the results
purchase_analysis_df=pd.DataFrame({"Number of unique items":[items],"Average Price":[average_price],"Number of Purchases":[purchases_number],
                    "Total Revenue":[revenue]})
#formatting the data 
purchase_analysis_df["Average Price"]=purchase_analysis_df["Average Price"].map("${:.2f}".format)
purchase_analysis_df["Total Revenue"]=purchase_analysis_df["Total Revenue"].map("${:,}".format)
purchase_analysis_df

,Number of unique items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
#grouping data frame by gender and counting unique values
gender=purchase_data_df.groupby('Gender').nunique()
#only selecting SN column to get count of male/female players
gender=gender["SN"]
#calculating the percentage
female=(gender[0]/players)*100
male =(gender[1]/players)*100
other=(gender[2]/players)*100
#creating new data frame to display demographics and formatting data
demographics_df=pd.DataFrame(gender)
demographics_df["Percentage of Players"]=[female, male, other]
demographics_df=demographics_df.rename(columns={"SN":"Total Count"})
demographics_df["Percentage of Players"]=demographics_df["Percentage of Players"].map("{:.2f}%".format)
demographics_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [8]:
#calculating the purchase count and total purchase value by using groupby
purchase_count =purchase_data_df.groupby('Gender')["Purchase ID"].nunique()
total_purchase=purchase_data_df.groupby('Gender')["Price"].sum()
#calculating average purchase price for each gender
avg_price_female=(total_purchase[0]/purchase_count[0])
avg_price_male=total_purchase[1]/purchase_count[1]
avg_price_other=total_purchase[2]/purchase_count[2]
#calculating average purchase price per person
price_perperson_female=(purchase_count[0]/gender[0])* avg_price_female
price_perperson_male=(purchase_count[1]/gender[1])* avg_price_male
price_perperson_other=(purchase_count[2]/gender[2])* avg_price_other

In [7]:
#creating data frame to display purchase analysis
purchase_by_gender_df=pd.DataFrame(purchase_count)
purchase_by_gender_df=purchase_by_gender_df.rename(columns={"Purchase ID":"Purchase Count"})
purchase_by_gender_df["Total Purchase Value"]=total_purchase
purchase_by_gender_df["Total Purchase Value"]=purchase_by_gender_df["Total Purchase Value"].map("${:,.2f}".format)
purchase_by_gender_df["Average Purchase Price"]=[avg_price_female,avg_price_male,avg_price_other]
purchase_by_gender_df["Average Total Purchase per Person"]=[price_perperson_female,price_perperson_male,price_perperson_other]
purchase_by_gender_df["Average Purchase Price"]=purchase_by_gender_df["Average Purchase Price"].map("${:.2f}".format)
purchase_by_gender_df["Average Total Purchase per Person"]=purchase_by_gender_df["Average Total Purchase per Person"].map("${:.2f}".format)
purchase_by_gender_df

,Purchase Count,Total Purchase Value,Average Purchase Price,Average Total Purchase per Person
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,"$1,967.64",$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56
